In [ ]:
# ทำ function สำหรับ model ไว้สำหรับ run backtest และให้เก็บข้อมูลออกมาในรูปของ [], ถ้าผลลัพธ์ออกมามันมี index ตัว ให้เอาแค่ตัว col 0 มา

# check vectorbt ว่าทำ multi asset ได้ไหม

In [2]:
import pandas as pd
import vectorbt as vbt
import pandas_ta as ta
import matplotlib.pyplot as plt
from tvDatafeed import TvDatafeed, Interval
import config_api

In [3]:
tv = TvDatafeed(config_api.tradingview_user, config_api.tradingview_password, chromedriver_path= None)

In [80]:
symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BNBUSDT', 'ADAUSDT'] 
start = '2020-01-01'
end = '2022-01-01'


In [360]:
btc = tv.get_hist(symbol= 'BTCUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars=5000)
eth = tv.get_hist(symbol= 'ETHUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars=5000)
xrp = tv.get_hist(symbol= 'XRPUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars=5000)
bnb = tv.get_hist(symbol= 'BNBUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars=5000)
ada = tv.get_hist(symbol= 'ADAUSDT', exchange= 'BINANCE', interval= Interval.in_daily, n_bars=5000)

In [288]:
ada.vbt.ohlcv.plot()

FigureWidget({
    'data': [{'close': array([0.2426 , 0.262  , 0.27004, ..., 1.048  , 1.059  , 1.05   ]),
    …

In [371]:
symbol = [btc, eth, xrp, bnb, ada]

In [377]:
for i in symbol:

    i = i.ta.rsi(length= 14, append= True)
for i in symbol:
    i = i.ta.ma_obv(fast=20 , slow= 40, ma_type= 'ema', append= True)

In [381]:
for i in symbol:
    i.loc[i['RSI_14'] > 50, 'rsi'] = 1
    i.loc[i['RSI_14'] < 50, 'rsi'] = 0
    i.loc[i['IBVe_20_40_diff'] > 0, 'vol_in'] = 1
    i.loc[i['IBVe_20_40_diff'] < 0, 'vol_in'] = 0



    i['signal'] = (i['rsi'] == 1) & (i['vol_in'] == 1)
    i['next_open'] = i['open'].shift(-1)

    i.ta.tsignals(i['signal'], asbool= True, append= True)

In [384]:
df = pd.concat(symbol, axis=1, keys= symbols)

In [394]:
df = df.dropna()

In [395]:
df

BTCUSDT                                          \
                              symbol      open      high       low     close   
datetime                                                                       
2018-06-12 07:00:00  BINANCE:BTCUSDT   6872.00   6890.51   6435.00   6530.00   
2018-06-13 07:00:00  BINANCE:BTCUSDT   6530.00   6627.60   6118.68   6292.78   
2018-06-14 07:00:00  BINANCE:BTCUSDT   6292.78   6720.00   6260.00   6635.98   
2018-06-15 07:00:00  BINANCE:BTCUSDT   6638.67   6666.66   6362.31   6388.90   
2018-06-16 07:00:00  BINANCE:BTCUSDT   6380.01   6560.00   6320.86   6483.98   
...                              ...       ...       ...       ...       ...   
2022-01-25 07:00:00  BINANCE:BTCUSDT  36660.35  37545.14  35701.00  36958.32   
2022-01-26 07:00:00  BINANCE:BTCUSDT  36958.32  38919.98  36234.63  36809.34   
2022-01-27 07:00:00  BINANCE:BTCUSDT  36807.24  37234.47  35507.01  37160.10   
2022-01-28 07:00:00  BINANCE:BTCUSDT  37160.11  38000.00  36155.01  37716.56   
2022-01-29 07:00:00  BINANCE:BTCUSDT  37716.57  38720.74  37268.44  38166.84   

                                                                          \
                           volume     RSI_14           OBV       OBVe_20   
datetime                                                                   
2018-06-12 07:00:00  36722.211191  29.092692 -3.238398e+05 -2.989238e+05   
2018-06-13 07:00:00  46185.518100  26.617681 -3.700253e+05 -3.056954e+05   
2018-06-14 07:00:00  44478.130837  35.206009 -3.255472e+05 -3.075860e+05   
2018-06-15 07:00:00  30018.329905  32.277219 -3.555655e+05 -3.121555e+05   
2018-06-16 07:00:00  22277.241147  34.534171 -3.332883e+05 -3.141681e+05   
...                           ...        ...           ...           ...   
2022-01-25 07:00:00  49225.222330  30.337932  3.217542e+06  3.233717e+06   
2022-01-26 07:00:00  69830.160360  29.979425  3.147712e+06  3.225526e+06   
2022-01-27 07:00:00  53020.432800  32.016385  3.200732e+06  3.223165e+06   
2022-01-28 07:00:00  42139.620920  35.235258  3.242872e+06  3.225042e+06   
2022-01-29 07:00:00  26128.111620  37.801588  3.269000e+06  3.229228e+06   

                                   ...       ADAUSDT                       \
                          OBVe_40  ...       OBVe_40 IBVe_20_40_diff  rsi   
datetime                           ...                                      
2018-06-12 07:00:00 -2.849885e+05  ...  2.303622e+07   -8.205861e+07  0.0   
2018-06-13 07:00:00 -2.891367e+05  ...  2.383192e+06   -9.391305e+07  0.0   
2018-06-14 07:00:00 -2.909128e+05  ... -7.443717e+06   -9.432791e+07  0.0   
2018-06-15 07:00:00 -2.940666e+05  ... -2.237135e+07   -9.956110e+07  0.0   
2018-06-16 07:00:00 -2.959798e+05  ... -3.272281e+07   -9.993762e+07  0.0   
...                           ...  ...           ...             ...  ...   
2022-01-25 07:00:00  3.319480e+06  ...  4.547301e+10   -4.125867e+08  0.0   
2022-01-26 07:00:00  3.311101e+06  ...  4.540637e+10   -4.367556e+08  0.0   
2022-01-27 07:00:00  3.305717e+06  ...  4.533397e+10   -4.641106e+08  0.0   
2022-01-28 07:00:00  3.302652e+06  ...  4.527182e+10   -4.791002e+08  0.0   
2022-01-29 07:00:00  3.301010e+06  ...  4.521707e+10   -4.856154e+08  0.0   

                                                                            \
                    vol_in signal next_open TS_Trends TS_Trades TS_Entries   
datetime                                                                     
2018-06-12 07:00:00    0.0  False   0.16309     False       0.0      False   
2018-06-13 07:00:00    0.0  False   0.15798     False       0.0      False   
2018-06-14 07:00:00    0.0  False   0.17091     False       0.0      False   
2018-06-15 07:00:00    0.0  False   0.16098     False       0.0      False   
2018-06-16 07:00:00    0.0  False   0.16209     False       0.0      False   
...                    ...    ...       ...       ...       ...        ...   
2022-01-25 07:00:00    0.0  False   1.04300     False       0.

In [385]:
port_btc = vbt.Portfolio.from_signals(
    df['BTCUSDT'].close,
    entries= df['BTCUSDT']['TS_Entries'],
    exits= df['BTCUSDT']['TS_Exits'],
    freq= '1D',
    init_cash= 100,
    fees= 0.0025,
    slippage= 0.0050
)
port_eth = vbt.Portfolio.from_signals(
    df['ETHUSDT'].close,
    entries= df['ETHUSDT']['TS_Entries'],
    exits= df['ETHUSDT']['TS_Exits'],
    freq= '1D',
    init_cash= 100,
    fees= 0.0025,
    slippage= 0.0050
)




In [396]:
df['ADAUSDT'].close

datetime
2018-06-12 07:00:00    0.16297
2018-06-13 07:00:00    0.15791
2018-06-14 07:00:00    0.17062
2018-06-15 07:00:00    0.16097
2018-06-16 07:00:00    0.16191
                        ...   
2022-01-25 07:00:00    1.04300
2022-01-26 07:00:00    1.07700
2022-01-27 07:00:00    1.04200
2022-01-28 07:00:00    1.04800
2022-01-29 07:00:00    1.05900
Freq: D, Name: close, Length: 1328, dtype: float64

In [397]:
port_xrp = vbt.Portfolio.from_signals(
    df['XRPUSDT'].close,
    entries= df['XRPUSDT']['TS_Entries'],
    exits= df['XRPUSDT']['TS_Exits'],
    freq= '1D',
    init_cash= 100,
    fees= 0.0025,
    slippage= 0.0050
)
port_bnb = vbt.Portfolio.from_signals(
    df['BNBUSDT'].close,
    entries= df['BNBUSDT']['TS_Entries'],
    exits= df['BNBUSDT']['TS_Exits'],
    freq= '1D',
    init_cash= 100,
    fees= 0.0025,
    slippage= 0.0050
)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type array(pyobject, 1d, C)[0m
[0m[1mDuring: typing of argument at C:\Anaconda3\lib\site-packages\vectorbt\portfolio\nb.py (3204)[0m
[1m
File "C:\Anaconda3\lib\site-packages\vectorbt\portfolio\nb.py", line 3204:[0m
[1mdef simulate_from_signals_nb(target_shape: tp.Shape,
    <source elided>
    ```"""
[1m    check_group_lens(group_lens, target_shape[1])
[0m    [1m^[0m[0m


In [388]:
port_ada = vbt.Portfolio.from_signals(
    df['ADAUSDT'].close,
    entries= df['ADAUSDT']['TS_Entries'],
    exits= df['ADAUSDT']['TS_Exits'],
    freq= '1D',
    init_cash= 100,
    fees= 0.0025,
    slippage= 0.0050
)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type array(pyobject, 1d, C)[0m
[0m[1mDuring: typing of argument at C:\Anaconda3\lib\site-packages\vectorbt\portfolio\nb.py (3204)[0m
[1m
File "C:\Anaconda3\lib\site-packages\vectorbt\portfolio\nb.py", line 3204:[0m
[1mdef simulate_from_signals_nb(target_shape: tp.Shape,
    <source elided>
    ```"""
[1m    check_group_lens(group_lens, target_shape[1])
[0m    [1m^[0m[0m


In [ ]:
df['XRPUSDT'].loc[df['XRPUSDT']['TS_Trades'] > 0]

In [399]:
port_eth.stats()

Start                         2017-08-17 07:00:00
End                           2022-01-30 07:00:00
Period                         1628 days 00:00:00
Start Value                                 100.0
End Value                                767.7514
Total Return [%]                         667.7514
Benchmark Return [%]                   768.238411
Max Gross Exposure [%]                      100.0
Total Fees Paid                         46.259453
Max Drawdown [%]                        64.347542
Max Drawdown Duration          1090 days 00:00:00
Total Trades                                   41
Total Closed Trades                            41
Total Open Trades                               0
Open Trade P&L                                0.0
Win Rate [%]                             39.02439
Best Trade [%]                          154.24268
Worst Trade [%]                        -25.899406
Avg Winning Trade [%]                   33.042654
Avg Losing Trade [%]                    -7.300077


In [313]:
print(['HEADER TITLE{}'.format(i) for i in symbols])

['HEADER TITLEBTCUSDT', 'HEADER TITLEETHUSDT', 'HEADER TITLEXRPUSDT', 'HEADER TITLEBNBUSDT', 'HEADER TITLEADAUSDT']


In [279]:
port.plot().show()

In [284]:
port.value().vbt.plot()

AttributeError: 'list' object has no attribute 'value'

In [244]:
ada.loc[ada['signal'] == 1]

,symbol,open,high,low,close,volume,RSI_14,OBV,OBVe_20,OBVe_40,IBVe_20_40_diff,rsi,vol_in,signal
datetime,,,,,,,,,,,,,,
2018-07-18 07:00:00,BINANCE:ADAUSDT,0.17136,0.19701,0.17109,0.18053,4.163099e+08,65.286414,4.634617e+08,-2.084280e+08,-2.138005e+08,5.372557e+06,1.0,1.0,True
2018-07-19 07:00:00,BINANCE:ADAUSDT,0.18061,0.19217,0.17149,0.18191,2.822322e+08,65.781004,7.456940e+08,-1.175592e+08,-1.669959e+08,4.943670e+07,1.0,1.0,True
2018-07-20 07:00:00,BINANCE:ADAUSDT,0.18164,0.18348,0.15900,0.16129,2.533846e+08,53.512404,4.923093e+08,-5.947651e+07,-1.348347e+08,7.535819e+07,1.0,1.0,True
2018-07-21 07:00:00,BINANCE:ADAUSDT,0.16112,0.16998,0.15388,0.16327,1.749686e+08,54.392026,6.672780e+08,9.738200e+06,-9.570726e+07,1.054455e+08,1.0,1.0,True
2018-07-22 07:00:00,BINANCE:ADAUSDT,0.16340,0.17920,0.16164,0.16916,1.893262e+08,56.998639,8.566042e+08,9.039210e+07,-4.925304e+07,1.396451e+08,1.0,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-04 07:00:00,BINANCE:ADAUSDT,2.96500,2.96700,2.80000,2.83000,1.922713e+08,62.715692,5.018593e+10,4.901225e+10,4.778938e+10,1.222871e+09,1.0,1.0,True
2021-09-05 07:00:00,BINANCE:ADAUSDT,2.83100,2.95600,2.81300,2.91000,1.690978e+08,64.713430,5.035503e+10,4.914014e+10,4.791454e+10,1.225601e+09,1.0,1.0,True
2021-09-06 07:00:00,BINANCE:ADAUSDT,2.91100,2.92700,2.80000,2.83300,1.869705e+08,61.308424,5.016806e+10,4.923803e+10,4.802446e+10,1.213571e+09,1.0,1.0,True


In [ ]:
df = pd.concat

In [245]:
df = pd.concat(symbol, join= 'inner', axis=1, keys= symbols)

In [254]:
df

BTCUSDT                                          \
                              symbol      open      high       low     close   
datetime                                                                       
2018-05-04 07:00:00  BINANCE:BTCUSDT   9750.00   9830.04   9520.85   9713.99   
2018-05-05 07:00:00  BINANCE:BTCUSDT   9714.00  10020.00   9682.00   9864.00   
2018-05-06 07:00:00  BINANCE:BTCUSDT   9863.99   9970.00   9417.03   9659.01   
2018-05-07 07:00:00  BINANCE:BTCUSDT   9661.02   9689.67   9181.00   9365.00   
2018-05-08 07:00:00  BINANCE:BTCUSDT   9365.00   9475.70   9060.54   9187.56   
...                              ...       ...       ...       ...       ...   
2022-01-26 07:00:00  BINANCE:BTCUSDT  36958.32  38919.98  36234.63  36809.34   
2022-01-27 07:00:00  BINANCE:BTCUSDT  36807.24  37234.47  35507.01  37165.72   
2022-01-28 07:00:00  BINANCE:BTCUSDT  37160.11  38000.00  36155.01  37716.56   
2022-01-29 07:00:00  BINANCE:BTCUSDT  37716.57  38720.74  37268.44  38169.00   
2022-01-30 07:00:00  BINANCE:BTCUSDT  38166.83  38359.26  37712.05  37897.10   

                                                                             \
                           volume           OBV       OBVe_20       OBVe_40   
datetime                                                                      
2018-05-04 07:00:00  28681.588879 -1.543626e+05 -1.792314e+05 -1.897669e+05   
2018-05-05 07:00:00  24990.018345 -1.293725e+05 -1.744830e+05 -1.868208e+05   
2018-05-06 07:00:00  27690.351559 -1.570629e+05 -1.728239e+05 -1.853692e+05   
2018-05-07 07:00:00  33787.640012 -1.908505e+05 -1.745407e+05 -1.856366e+05   
2018-05-08 07:00:00  25533.831889 -2.163844e+05 -1.785258e+05 -1.871365e+05   
...                           ...           ...           ...           ...   
2022-01-26 07:00:00  69830.160360  3.147720e+06  3.225528e+06  3.311102e+06   
2022-01-27 07:00:00  53017.504930  3.200737e+06  3.223167e+06  3.305718e+06   
2022-01-28 07:00:00  42153.193590  3.242890e+06  3.225045e+06  3.302653e+06   
2022-01-29 07:00:00  26124.847060  3.269015e+06  3.229233e+06  3.301013e+06   
2022-01-30 07:00:00  14063.808750  3.254951e+06  3.231682e+06  3.298766e+06   

                                     ...  ADAUSDT                           \
                    IBVe_20_40_diff  ...    close        volume     RSI_14   
datetime                             ...                                     
2018-05-04 07:00:00    10535.470019  ...  0.35800  5.606927e+07  67.422879   
2018-05-05 07:00:00    12337.869581  ...  0.36502  4.696269e+07  68.339455   
2018-05-06 07:00:00    12545.317357  ...  0.34654  6.392687e+07  63.291119   
2018-05-07 07:00:00    11095.877092  ...  0.33046  6.406094e+07  59.193605   
2018-05-08 07:00:00     8610.659879  ...  0.32213  4.510501e+07  57.130189   
...                             ...  ...      ...           ...        ...   
2022-01-26 07:00:00   -85574.052679  ...  1.07700  3.365040e+08  40.797844   
2022-01-27 07:00:00   -82551.426523  ...  1.04200  1.847625e+08  39.275267   
2022-01-28 07:00:00   -77608.221609  ...  1.04800  1.376901e+08  39.690789   
2022-01-29 07:00:00   -71779.717996  ...  1.05900  8.953128e+07  40.494703   
2022-01-30 07:00:00   -67083.448892  ...  1.05100  4.027451e+07  40.076299   

                                                                               \
                              OBV       OBVe_20       OBVe_40 IBVe_20_40_diff   
datetime                                                                        
2018-05-04 07:00:00  3.578943e+08           NaN           NaN             NaN   
2018-05-05 07:00:00  4.048570e+08           NaN           NaN             NaN   
2018-05-06 07:00:00  3.409301e+08  2.398182e+08           NaN             NaN   
2018-05-07 07:00:00  2.768692e+08  2.433469e+08           NaN             NaN   
2018-05-08 07:00:00  2.317642e+08  2.422438e+08           NaN             NaN   
...                           ...           ...           ..

In [ ]:
df.loc[df['BTCUSDT']['signal'] == 1]

In [262]:
df = df.dropna()